In [ ]:
"""

Apply the uncertainty of the parameters from Burgard et al. 2022 to show that 
uncertainty from parameters is lower than uncertainty from parameterisations

"""


In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import cartopy
import cartopy.crs as ccrs
import matplotlib as mpl
import pandas as pd 
from tqdm.notebook import trange, tqdm
import time
import multimelt.melt_functions as meltf
import multimelt.plume_functions as pf
import summer_paper.useful_functions as uf
from multimelt.constants import *
import seaborn as sns
import os, glob
import distributed

In [ ]:
#home_path = '/Users/claraburgard/bettik_clara/'
home_path='/bettik/burgardc/'
#home_path='/data/users/burgardc/bettik_clara/'

In [ ]:
nemo_run = 'OPM021'

In [ ]:
inputpath_data=home_path+'/DATA/BASAL_MELT_PARAM/interim/NEMO_eORCA025.L121_'+nemo_run+'_ANT_STEREO/'
inputpath_mask = home_path+'/DATA/BASAL_MELT_PARAM/interim/ANTARCTICA_IS_MASKS/nemo_5km_'+nemo_run+'/'
inputpath_profiles = home_path+'/DATA/BASAL_MELT_PARAM/interim/T_S_PROF/nemo_5km_'+nemo_run+'/'
outputpath_simple = home_path+'/DATA/BASAL_MELT_PARAM/interim/SIMPLE/nemo_5km_'+nemo_run+'/'
outputpath_simple_all = home_path+'/DATA/BASAL_MELT_PARAM/interim/SIMPLE/nemo_5km_06161821_oneFRIS/'
inputpath_plumes = home_path+'/DATA/BASAL_MELT_PARAM/interim/PLUMES/nemo_5km_'+nemo_run+'/'
outputpath_plumes = home_path+'/DATA/BASAL_MELT_PARAM/interim/PLUMES/nemo_5km_'+nemo_run+'/'
inputpath_boxes = home_path+'/DATA/BASAL_MELT_PARAM/interim/BOXES/nemo_5km_'+nemo_run+'/'
outputpath_boxes = home_path+'/DATA/BASAL_MELT_PARAM/interim/BOXES/nemo_5km_'+nemo_run+'/'
outputpath_melt = home_path+'/DATA/SUMMER_PAPER/processed/OCEAN_MELT_RATE_BT_RESULTS/nemo_5km_'+nemo_run+'/'
outputpath = home_path+'/DATA/BASAL_MELT_PARAM/interim/'
plot_path = home_path+'/PLOTS/first-look/'

In [ ]:
# make the domain a little smaller to make the computation even more efficient - file isf has already been made smaller at its creation
map_lim = [-3000000,3000000]

In [ ]:
file_other = xr.open_dataset(inputpath_data+'corrected_draft_bathy_isf.nc')#, chunks={'x': chunk_size, 'y': chunk_size})
file_other_cut = uf.cut_domain_stereo(file_other, map_lim, map_lim)
file_conc = xr.open_dataset(inputpath_data+'isfdraft_conc_Ant_stereo.nc')
file_conc_cut = uf.cut_domain_stereo(file_conc, map_lim, map_lim)

In [ ]:
#file_isf_orig = xr.open_dataset(inputpath_mask+'nemo_5km_isf_masks_and_info_and_distance_new.nc')
file_isf_orig = xr.open_dataset(inputpath_mask+'nemo_5km_isf_masks_and_info_and_distance_new_oneFRIS.nc')
nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
large_isf = file_isf_nonnan['Nisf'].where(file_isf_nonnan['isf_area_here'] >= 2500, drop=True)
file_isf = file_isf_nonnan.sel(Nisf=large_isf)
file_isf['isf_name'] = file_isf['isf_name'].astype(str)
file_isf['region'] = file_isf['region'].astype(str)


#file_TS = xr.open_dataset(inputpath_profiles+'T_S_mean_prof_km_contshelf_1980-2018.nc')
#file_TS = file_TS.rename({'dist_from_front': 'profile_domain'})
#file_TS_orig = xr.open_dataset(inputpath_profiles+'T_S_mean_prof_corrected_km_contshelf_and_offshore_1980-2018.nc')
file_TS_orig = xr.open_dataset(inputpath_profiles+'T_S_mean_prof_corrected_km_contshelf_and_offshore_1980-2018_oneFRIS.nc')
file_TS = file_TS_orig.sel(Nisf=large_isf).sel(profile_domain=[50,1000])#.isel(time=range(40,70))

In [ ]:
lon = file_isf.longitude
lat = file_isf.latitude

In [ ]:
file_isf_conc = file_conc_cut['isfdraft_conc']

xx = file_isf.x
yy = file_isf.y
dx = (xx[2] - xx[1]).values
dy = (yy[2] - yy[1]).values
grid_cell_area = abs(dx*dy)  
grid_cell_area_weighted = file_isf_conc * grid_cell_area

In [ ]:
ice_draft_pos = file_other_cut['corrected_isfdraft']
ice_draft_neg = -ice_draft_pos

In [ ]:
isf_stack_mask = uf.create_stacked_mask(file_isf['ISF_mask'], file_isf.Nisf, ['y','x'], 'mask_coord')

In [ ]:
box_charac_all_2D = xr.open_dataset(inputpath_boxes + 'nemo_5km_boxes_2D_oneFRIS.nc')
box_charac_all_1D = xr.open_dataset(inputpath_boxes + 'nemo_5km_boxes_1D_oneFRIS.nc')
plume_charac = xr.open_dataset(inputpath_plumes+'nemo_5km_plume_characteristics_oneFRIS.nc')

In [ ]:
param_var_of_int_2D = file_isf[['ISF_mask', 'latitude', 'longitude', 'dGL']]
param_var_of_int_1D = file_isf[['front_bot_depth_avg', 'front_bot_depth_max','isf_name']]

In [ ]:
geometry_info_2D = plume_charac.merge(param_var_of_int_2D).merge(ice_draft_pos).rename({'corrected_isfdraft':'ice_draft_pos'}).merge(grid_cell_area_weighted).rename({'isfdraft_conc':'grid_cell_area_weighted'}).merge(file_isf_conc)
geometry_info_1D = param_var_of_int_1D

In [ ]:
geometry_info_2D = geometry_info_2D.assign_coords({'option': ['cavity','lazero','local']})

SIMPLE

In [ ]:
gammas = xr.DataArray(np.array([[1.8e-6,2.4e-6,2.7e-6,3.0e-6,9.0e-6],
                                [8.7e-5*2.9e-3,11.1e-5*2.9e-3,12.0e-5*2.9e-3,12.8e-5*2.9e-3,14.5e-5*2.9e-3],
                               [6.0e-5,7.4e-5,7.9e-5,8.2e-5,9.2e-5],
                               [10.1e-5*2.9e-3,12.9e-5*2.9e-3,14.0e-5*2.9e-3,15.1e-5*2.9e-3,16.7e-5*2.9e-3],
                               [7.3e-5,8.9e-5,9.3e-5,9.7e-5,11.0e-5]]), dims={'param': ['linear_local', 'quadratic_local', 'quadratic_local_locslope', 'quadratic_mixed_mean', 'quadratic_mixed_locslope'],'pctl': [5,33,50,66,95]})
gammas = gammas.assign_coords({'pctl': [5,33,50,66,95], 'param': ['linear_local', 'quadratic_local', 'quadratic_local_locslope', 'quadratic_mixed_mean', 'quadratic_mixed_locslope']})


In [ ]:
simple_param_types = ['linear_local', 'quadratic_local', 'quadratic_local_locslope',
                      'quadratic_mixed_mean', 'quadratic_mixed_locslope'] 

nisf_list = geometry_info_1D.Nisf
T_S_profile = file_TS.ffill(dim='depth')

#for pctl in tqdm([5,33,66,95]):
for pctl in tqdm([50]):
    
    for mparam in simple_param_types:

        gamma = gammas.sel(param=mparam,pctl=pctl)

        eval_1D = meltf.calculate_melt_rate_Gt_and_box1_all_isf(nisf_list, T_S_profile, 
                                                                geometry_info_2D, geometry_info_1D, isf_stack_mask, 
                                                                mparam, gamma, U_param=True, 
                                                                box_charac_2D=box_charac_all_2D, box_charac_1D=box_charac_all_1D, verbose=False)

        eval_1D.to_netcdf(outputpath_melt+'eval_metrics_1D_'+mparam+'_oneFRIS_'+str(pctl).zfill(2)+'.nc')


In [ ]:
gammas_plumes_boxes = xr.DataArray(np.array([[1.4e-4,2.2e-4,2.6e-4,3.1e-4,6.0e-4],
                                            [0.22e-5,0.39e-5,0.44e-5,0.76e-5,1.99e-5]]), 
                                   dims={'param': ['lazero19', 'boxes_4_pismyes_picopno'],'pctl': [5,33,50,66,95]})
gammas_plumes_boxes = gammas_plumes_boxes.assign_coords({'param': ['lazero19', 'boxes_4_pismyes_picopno'],'pctl': [5,33,50,66,95]})


In [ ]:
nisf_list = geometry_info_1D.Nisf
T_S_profile = file_TS.ffill(dim='depth')

#for pctl in tqdm([5,33,66,95]):
for pctl in tqdm([50]):
    
    mparam = 'lazero19'
        
    gamma = gammas_plumes_boxes.sel(param=mparam,pctl=pctl)
    E0 = ((12.3 / (gamma*10**4)**2) + 0.6) * 10**(-2)
        
    eval_1D = meltf.calculate_melt_rate_Gt_and_box1_all_isf(nisf_list, T_S_profile, 
                                                         geometry_info_2D, geometry_info_1D, isf_stack_mask, 
                                                         mparam, gamma, E0=E0, 
                                                         box_charac_2D=box_charac_all_2D, box_charac_1D=box_charac_all_1D, verbose=False)
    
    eval_1D.to_netcdf(outputpath_melt+'eval_metrics_1D_'+mparam+'_oneFRIS_'+str(pctl).zfill(2)+'.nc')


In [ ]:
nisf_list = geometry_info_1D.Nisf
T_S_profile = file_TS.ffill(dim='depth')

#for pctl in tqdm([5,33,66,95]):
for pctl in tqdm([50]):   
    mparam = 'boxes_4_pismyes_picopno'
    pism_version = 'yes'
    nD_config = 4
        
    gamma = gammas_plumes_boxes.sel(param=mparam,pctl=pctl)
    C = ((4.8 / (gamma*10**5)**2) - 2.4) * 10**6
    if C < 0.1e6:
        print('low C')
        C = 0.1e6
        
    eval_1D = meltf.calculate_melt_rate_Gt_and_box1_all_isf(nisf_list, T_S_profile, 
                                                             geometry_info_2D, geometry_info_1D, isf_stack_mask, 
                                                             mparam, gamma, C=C, angle_option='local',
                                                             box_charac_2D=box_charac_all_2D, box_charac_1D=box_charac_all_1D, box_tot=nD_config, 
                                                             box_tot_option='nD_config', pism_version=pism_version, picop_opt='no',
                                                             verbose=False)  
    
    eval_1D.to_netcdf(outputpath_melt+'eval_metrics_1D_'+mparam+'_oneFRIS_'+str(pctl).zfill(2)+'.nc')
